In [1]:
import os
import sys

path = os.getcwd()
while os.path.basename(os.path.normpath(path)) != 'roman-pandeia':
    path = os.path.dirname(path)
repo_path = path
if repo_path not in sys.path:
    sys.path.append(repo_path)

In [2]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors
from glob import glob
import multiprocessing
import pandas as pd
from tqdm import tqdm
import time
from copy import deepcopy
from multiprocessing import Pool
from hydra import initialize, compose
import pickle

from package.helpers.test_physical_lens import TestPhysicalLens
from package.helpers.lens import Lens
from package.plots import diagnostic_plot, plot
from package.utils import util
from package.scripts import generate
from package.helpers import pyhalo


In [3]:
def get_model(lens):
    return lens.get_array(num_pix=51, side=5.61)

In [4]:
with initialize(version_base=None, config_path='../../config'):
    config = compose(config_name='config.yaml')  # overrides=['machine=uzay']

array_dir, data_dir, repo_dir, pickle_dir = config.machine.array_dir, config.machine.data_dir, config.machine.repo_dir, config.machine.pickle_dir
array_dir = os.path.join(array_dir, 'skypy_output')
util.create_directory_if_not_exists(array_dir)

In [5]:
# open pickled lens list
with open(os.path.join(pickle_dir, 'skypy_output_lens_list_with_subhalos'), 'rb') as results_file:
    lens_list = pickle.load(results_file)

In [6]:
# split up the lenses into batches based on core count
cpu_count = multiprocessing.cpu_count()
process_count = int(cpu_count / 2)  # TODO consider making this larger, but using all CPUs has crashed
generator = util.batch_list(lens_list, process_count)
batches = list(generator)

In [11]:
i = 0

# process the batches
for batch in tqdm(batches):
    pool = Pool(processes=process_count) 
    for output in pool.map(get_model, batch):
        (model) = output
        np.save(os.path.join(array_dir, f'skypy_output_{str(i).zfill(8)}.npy'), model)
        i += 1

100%|██████████| 5/5 [00:04<00:00,  1.22it/s]
